# IIC-3670 NLP UC

- Versiones de librerías, python 3.8.10

- numpy 1.20.3
- flair 0.12
- allennlp 0.9.0

## Actividad en clase

Vamos a entrenar con el **transformer seq2seq** que vimos en clases. Siga la definición de la arquitectura del transformer haciendo lo siguiente:

- Usando los mismos hiperparámetros de clases, entrene a 50 epochs y mida validation loss. Este modelo se llama transformer large.
- Sobre los mismos datos anteriores, disminuya el número de layers a 4, reinicialize el optimizer, entrene a 50 epochs y mida validation loss. Este modelo se llama transformer base.
- Usando los mismos hiperparámetros de clases, aumente max_seq_length al doble, reinicialize el optimzer, entrene a 50 epochs y mida validation loss. Observe que debe generar las secuencias de nuevo. Este modelo se llama transformer memory.
- Compare transformer base con transformer large. ¿Qué ocurre cuando aumentamos el número de layers?
- Compare transformer base con transformer memory. ¿Qué ocurre cuando aumentamos el largo de la entrada?
- Cuanto termine, me avisa para entregarle una **L (logrado)**.
- Recuerde que las L otorgan un bono en la nota final de la asignatura.


***Tiene hasta el final de la clase.***

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy

In [2]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        # Ensure that the model dimension (d_model) is divisible by the number of heads
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        # Initialize dimensions
        self.d_model = d_model # Model's dimension
        self.num_heads = num_heads # Number of attention heads
        self.d_k = d_model // num_heads # Dimension of each head's key, query, and value
        
        # Linear layers for transforming inputs
        self.W_q = nn.Linear(d_model, d_model) # Query transformation
        self.W_k = nn.Linear(d_model, d_model) # Key transformation
        self.W_v = nn.Linear(d_model, d_model) # Value transformation
        self.W_o = nn.Linear(d_model, d_model) # Output transformation
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        # Calculate attention scores
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        
        # Apply mask if provided (useful for preventing attention to certain parts like padding)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        
        # Softmax is applied to obtain attention probabilities
        attn_probs = torch.softmax(attn_scores, dim=-1)
        
        # Multiply by values to obtain the final output
        output = torch.matmul(attn_probs, V)
        return output
        
    def split_heads(self, x):
        # Reshape the input to have num_heads for multi-head attention
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        
    def combine_heads(self, x):
        # Combine the multiple heads back to original shape
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        
    def forward(self, Q, K, V, mask=None):
        # Apply linear transformations and split heads
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        
        # Perform scaled dot-product attention
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        
        # Combine heads and apply output transformation
        output = self.W_o(self.combine_heads(attn_output))
        return output

In [3]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

In [4]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()
        
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe.unsqueeze(0))
        
    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

In [5]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

In [6]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

In [7]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output

In [8]:
src_vocab_size = 5000
tgt_vocab_size = 5000
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 100
dropout = 0.1

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

# Generate random sample data
src_data = torch.randint(1, src_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)
tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)

In [10]:
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(50):
    optimizer.zero_grad()
    output = transformer(src_data, tgt_data[:, :-1])
    loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Epoch: 1, Loss: 8.690889358520508
Epoch: 2, Loss: 8.556205749511719
Epoch: 3, Loss: 8.482999801635742
Epoch: 4, Loss: 8.42567253112793
Epoch: 5, Loss: 8.360939979553223
Epoch: 6, Loss: 8.300979614257812
Epoch: 7, Loss: 8.223347663879395
Epoch: 8, Loss: 8.134797096252441
Epoch: 9, Loss: 8.05554485321045
Epoch: 10, Loss: 7.973031044006348
Epoch: 11, Loss: 7.889521598815918
Epoch: 12, Loss: 7.808218955993652
Epoch: 13, Loss: 7.728190898895264
Epoch: 14, Loss: 7.6423211097717285
Epoch: 15, Loss: 7.564476013183594
Epoch: 16, Loss: 7.4857497215271
Epoch: 17, Loss: 7.400502681732178
Epoch: 18, Loss: 7.313865661621094
Epoch: 19, Loss: 7.234110355377197
Epoch: 20, Loss: 7.156230449676514
Epoch: 21, Loss: 7.0781474113464355
Epoch: 22, Loss: 6.989865779876709
Epoch: 23, Loss: 6.9126296043396
Epoch: 24, Loss: 6.837189197540283
Epoch: 25, Loss: 6.765042781829834
Epoch: 26, Loss: 6.683589458465576
Epoch: 27, Loss: 6.613943576812744
Epoch: 28, Loss: 6.550961971282959
Epoch: 29, Loss: 6.47526645660400

In [11]:
transformer.eval()

# Generate random sample validation data
val_src_data = torch.randint(1, src_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)
val_tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)

with torch.no_grad():

    val_output = transformer(val_src_data, val_tgt_data[:, :-1])
    val_loss = criterion(val_output.contiguous().view(-1, tgt_vocab_size), val_tgt_data[:, 1:].contiguous().view(-1))
    print(f"Validation Loss: {val_loss.item()}")

Validation Loss: 8.731040000915527


In [16]:
num_layers = 4

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

In [17]:
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(50):
    optimizer.zero_grad()
    output = transformer(src_data, tgt_data[:, :-1])
    loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Epoch: 1, Loss: 8.69483757019043
Epoch: 2, Loss: 8.587047576904297
Epoch: 3, Loss: 8.509394645690918
Epoch: 4, Loss: 8.452327728271484
Epoch: 5, Loss: 8.398691177368164
Epoch: 6, Loss: 8.347383499145508
Epoch: 7, Loss: 8.290900230407715
Epoch: 8, Loss: 8.22274398803711
Epoch: 9, Loss: 8.157305717468262
Epoch: 10, Loss: 8.086456298828125
Epoch: 11, Loss: 8.01663875579834
Epoch: 12, Loss: 7.955038547515869
Epoch: 13, Loss: 7.882924556732178
Epoch: 14, Loss: 7.81955099105835
Epoch: 15, Loss: 7.75192928314209
Epoch: 16, Loss: 7.679123878479004
Epoch: 17, Loss: 7.608169078826904
Epoch: 18, Loss: 7.546481132507324
Epoch: 19, Loss: 7.467465400695801
Epoch: 20, Loss: 7.398152828216553
Epoch: 21, Loss: 7.329608917236328
Epoch: 22, Loss: 7.2537922859191895
Epoch: 23, Loss: 7.1818766593933105
Epoch: 24, Loss: 7.111588954925537
Epoch: 25, Loss: 7.0398430824279785
Epoch: 26, Loss: 6.965572357177734
Epoch: 27, Loss: 6.898441791534424
Epoch: 28, Loss: 6.829673767089844
Epoch: 29, Loss: 6.755025386810

In [18]:
transformer.eval()

with torch.no_grad():

    val_output = transformer(val_src_data, val_tgt_data[:, :-1])
    val_loss = criterion(val_output.contiguous().view(-1, tgt_vocab_size), val_tgt_data[:, 1:].contiguous().view(-1))
    print(f"Validation Loss: {val_loss.item()}")

Validation Loss: 8.746835708618164


In [19]:
num_layers = 6
max_seq_length = 200

# Generate random sample data
src_data = torch.randint(1, src_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)
tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

In [20]:
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(50):
    optimizer.zero_grad()
    output = transformer(src_data, tgt_data[:, :-1])
    loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Epoch: 1, Loss: 8.68437671661377
Epoch: 2, Loss: 8.585718154907227
Epoch: 3, Loss: 8.536176681518555
Epoch: 4, Loss: 8.500722885131836
Epoch: 5, Loss: 8.462150573730469
Epoch: 6, Loss: 8.41718864440918
Epoch: 7, Loss: 8.367278099060059
Epoch: 8, Loss: 8.31430435180664
Epoch: 9, Loss: 8.259011268615723
Epoch: 10, Loss: 8.206537246704102
Epoch: 11, Loss: 8.156159400939941
Epoch: 12, Loss: 8.101131439208984
Epoch: 13, Loss: 8.040860176086426
Epoch: 14, Loss: 7.992099285125732
Epoch: 15, Loss: 7.937398910522461
Epoch: 16, Loss: 7.873149871826172
Epoch: 17, Loss: 7.816011905670166
Epoch: 18, Loss: 7.763660430908203
Epoch: 19, Loss: 7.704605579376221
Epoch: 20, Loss: 7.649529457092285
Epoch: 21, Loss: 7.593634605407715
Epoch: 22, Loss: 7.533790111541748
Epoch: 23, Loss: 7.474223613739014
Epoch: 24, Loss: 7.41947603225708
Epoch: 25, Loss: 7.364382743835449
Epoch: 26, Loss: 7.3106608390808105
Epoch: 27, Loss: 7.257026672363281
Epoch: 28, Loss: 7.200265884399414
Epoch: 29, Loss: 7.1494126319885

In [21]:
transformer.eval()

# Generate random sample validation data
val_src_data = torch.randint(1, src_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)
val_tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)

with torch.no_grad():

    val_output = transformer(val_src_data, val_tgt_data[:, :-1])
    val_loss = criterion(val_output.contiguous().view(-1, tgt_vocab_size), val_tgt_data[:, 1:].contiguous().view(-1))
    print(f"Validation Loss: {val_loss.item()}")

Validation Loss: 8.715140342712402
